In [9]:
# import basic packages 
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [10]:
# 밴딧의 손잡이 목록
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [11]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
loss = -(tf.log(responsible_output) * reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [8]:
total_episodes = 1000
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)
        
        reward = pullBandit(bandit_arms[action])
        
        _, resp, ww = sess.run([update, responsible_output, weights], \
                              feed_dict={reward_holder:[reward], action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        
        i += 1
        
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [1. 0. 0. 0.]
Running reward for the 4 arms of the bandit: [-8. -2.  7. 12.]
Running reward for the 4 arms of the bandit: [-11.  -3.  13.  22.]
Running reward for the 4 arms of the bandit: [-15.  -5.  16.  41.]
Running reward for the 4 arms of the bandit: [-13.  -6.  14.  52.]
Running reward for the 4 arms of the bandit: [-11.  -2.  16.  62.]
Running reward for the 4 arms of the bandit: [-10.   2.  19.  78.]
Running reward for the 4 arms of the bandit: [-10.   1.  18.  96.]
Running reward for the 4 arms of the bandit: [ -9.   6.  18. 110.]
Running reward for the 4 arms of the bandit: [ -9.   2.  25. 127.]
Running reward for the 4 arms of the bandit: [ -5.   0.  25. 137.]
Running reward for the 4 arms of the bandit: [ -2.   3.  29. 153.]
Running reward for the 4 arms of the bandit: [ -7.   2.  26. 164.]
Running reward for the 4 arms of the bandit: [ -8.   2.  23. 180.]
Running reward for the 4 arms of the bandit: [-14.   0.  19. 194.]
Running